In [1]:
import torch
from torch import nn
from transformers import ViTImageProcessor, ViTForImageClassification
from torchinfo import summary

c:\Users\ganes\anaconda3\envs\TorchEnv2\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224')
model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')

In [3]:
# summary(model.vit.encoder.layer[0].attention.attention)

In [4]:
# import inspect

# print(inspect.getsource(model.__init__))

In [5]:
# import inspect

# print(inspect.getsource(model.forward))

In [6]:
# model.vit.embeddings.patch_embeddings.projection

In [7]:
# model.config

In [8]:
new_first_layer = nn.Conv2d(
    in_channels=16,
    out_channels=768,   
    kernel_size=16,
    stride=16,
    padding=0
)

with torch.no_grad():
    new_first_layer.weight[:, :3] = model.vit.embeddings.patch_embeddings.projection.weight.clone()
    new_first_layer.weight[:, 3:] = torch.mean(model.vit.embeddings.patch_embeddings.projection.weight, dim=1, keepdim=True).repeat(1, 13, 1, 1)

model.vit.embeddings.patch_embeddings.projection = new_first_layer
model.classifier = nn.Linear(in_features=768, out_features=1, bias=True)

model.config.num_channels = 16
model.config.num_labels = 1 
model.vit.embeddings.patch_embeddings.num_channels = 16

In [9]:
# model.config

In [10]:
# model.vit.embeddings.patch_embeddings

In [11]:
summary(model=model,
        input_size=(4, 16, 224, 224),
        col_names=['input_size', 'output_size', 'num_params', 'trainable'],
        col_width=20,
        row_settings=['var_names'])

Layer (type (var_name))                                                Input Shape          Output Shape         Param #              Trainable
ViTForImageClassification (ViTForImageClassification)                  [4, 16, 224, 224]    [4, 1]               --                   True
├─ViTModel (vit)                                                       [4, 16, 224, 224]    [4, 197, 768]        --                   True
│    └─ViTEmbeddings (embeddings)                                      [4, 16, 224, 224]    [4, 197, 768]        152,064              True
│    │    └─ViTPatchEmbeddings (patch_embeddings)                      [4, 16, 224, 224]    [4, 196, 768]        3,146,496            True
│    │    └─Dropout (dropout)                                          [4, 197, 768]        [4, 197, 768]        --                   --
│    └─ViTEncoder (encoder)                                            [4, 197, 768]        [4, 197, 768]        --                   True
│    │    └─ModuleList (